In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, pyll
import time
import colorama
import pickle
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import CatBoostClassifier, Pool
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

In [ ]:
train_df = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv")
test_df = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv")
sub_df = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/sample_submission.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape,test_df.shape

In [ ]:
train_df.isnull().sum()/len(train_df)

In [ ]:
test_df.isnull().sum()/len(test_df)

In [ ]:
train_df.target.value_counts()/len(train_df)

In [ ]:
train_df.dtypes

In [ ]:
train_df.head()

In [ ]:
train_df.gender.value_counts()

In [ ]:
train_df.enrolled_university.value_counts()

In [ ]:
train_df.major_discipline.value_counts()

In [ ]:
train_df.education_level.value_counts()

In [ ]:
train_df.experience.value_counts()

In [ ]:
train_df.company_size.value_counts()

In [ ]:
train_df.company_type.value_counts()

In [ ]:
train_df.last_new_job.value_counts()

****replacing special chars(like >,+) with numbers**

In [ ]:
train_df.experience.replace('>20','21',inplace=True)
train_df.experience.replace('<1','0',inplace=True)

In [ ]:
test_df.experience.replace('>20','21',inplace=True)
test_df.experience.replace('<1','0',inplace=True)

In [ ]:
train_df.experience=pd.to_numeric(train_df.experience)
test_df.experience=pd.to_numeric(test_df.experience)

In [ ]:
train_df['experience']=np.where(train_df['experience']>10,
                'Senior-level', np.where(train_df['experience']>3,
                                 'Intermediate-level' ,'Entry-Level'))

In [ ]:
test_df['experience']=np.where(test_df['experience']>10,
                'Senior-level', np.where(test_df['experience']>3,
                                 'Intermediate-level' ,'Entry-Level'))

In [ ]:
# here we are creating a class interval for each level of experience
train_df['experience'].value_counts()

In [ ]:
test_df['experience'].value_counts()

##### replacing special chars(like >,+) with numbers

In [ ]:
train_df.company_size.replace('<10','9',inplace=True)
train_df.company_size.replace('10/49','20',inplace=True)
train_df.company_size.replace('50-99','55',inplace=True)
train_df.company_size.replace('100-500','300',inplace=True)
train_df.company_size.replace('10000+','10001',inplace=True)
train_df.company_size.replace('500-999','600',inplace=True)
train_df.company_size.replace('5000-9999','6000',inplace=True)
train_df.company_size.replace('1000-4999','3000',inplace=True)

In [ ]:
test_df.company_size.replace('<10','9',inplace=True)
test_df.company_size.replace('10/49','20',inplace=True)
test_df.company_size.replace('50-99','55',inplace=True)
test_df.company_size.replace('100-500','300',inplace=True)
test_df.company_size.replace('10000+','10001',inplace=True)
test_df.company_size.replace('500-999','600',inplace=True)
test_df.company_size.replace('5000-9999','6000',inplace=True)
test_df.company_size.replace('1000-4999','3000',inplace=True)

In [ ]:
train_df.company_size= pd.to_numeric(train_df.company_size)
test_df.company_size= pd.to_numeric(test_df.company_size)

In [ ]:
train_df['company_size']=np.where(train_df['company_size']>2000,
          'Large-org.', np.where(train_df['company_size']>1,
                     'Small & Medium-org.','Undefined'))

In [ ]:
test_df['company_size']=np.where(test_df['company_size']>2000,
          'Large-org.', np.where(test_df['company_size']>1,
                     'Small & Medium-org.','Undefined'))

In [ ]:
test_df['company_size'].value_counts()

In [ ]:
train_df['company_size'].value_counts()

In [ ]:
train_df.last_new_job.replace('>4','5',inplace=True) 
train_df.last_new_job.replace('never','0',inplace=True)
test_df.last_new_job.replace('>4','5',inplace=True) 
test_df.last_new_job.replace('never','0',inplace=True)

In [ ]:
train_df.last_new_job= pd.to_numeric(train_df.last_new_job)

In [ ]:
test_df.last_new_job= pd.to_numeric(test_df.last_new_job)

In [ ]:
train_df['last_new_job']=np.where(train_df['last_new_job']>4,
                'Long-Time', np.where(train_df['last_new_job']>2,
                     'Medium-Time',np.where(train_df['last_new_job']>0 
                              ,'Latest', 'Never')))

In [ ]:
test_df['last_new_job']=np.where(test_df['last_new_job']>4,
                'Long-Time', np.where(test_df['last_new_job']>2,
                     'Medium-Time',np.where(test_df['last_new_job']>0 
                              ,'Latest', 'Never')))

In [ ]:
train_df.last_new_job.value_counts()

In [ ]:
test_df.last_new_job.value_counts()

In [ ]:
most_freq = 'Unknown'

In [ ]:
train_df['gender'] = train_df['gender'].fillna(most_freq)
test_df['gender'] = test_df['gender'].fillna(most_freq)

In [ ]:
train_df["enrolled_university"] = train_df["enrolled_university"].fillna(train_df["enrolled_university"].mode()[0])
test_df["enrolled_university"] = test_df["enrolled_university"].fillna(test_df["enrolled_university"].mode()[0])

In [ ]:
most_freq1 = "Unknown" 

In [ ]:
train_df["major_discipline"] = train_df['major_discipline'].fillna(most_freq1)
test_df["major_discipline"] = test_df['major_discipline'].fillna(most_freq1)

In [ ]:
train_df["education_level"] = train_df["education_level"].fillna(train_df["education_level"].mode()[0])
test_df["education_level"] = test_df["education_level"].fillna(test_df["education_level"].mode()[0])

In [ ]:
most_freq2 = "Entry-Level"

In [ ]:
train_df["experience"] = train_df["experience"].fillna(most_freq2)
test_df["experience"] = test_df["experience"].fillna(most_freq2)

In [ ]:
most_freq3 = "Undefined"

In [ ]:
train_df["company_size"] = train_df['company_size'].fillna(most_freq3)
test_df["company_size"] = test_df['company_size'].fillna(most_freq3)

In [ ]:
train_df["company_size"].value_counts()

In [ ]:
train_df["company_type"].value_counts()

In [ ]:
most_freq4 = "Other"

In [ ]:
train_df["company_type"] = train_df['company_type'].fillna(most_freq4)
test_df["company_type"] = test_df['company_type'].fillna(most_freq4)

In [ ]:
most_freq5 = "Unknown"

In [ ]:
train_df["last_new_job"] = train_df['last_new_job'].fillna(most_freq5)
test_df["last_new_job"] = test_df['last_new_job'].fillna(most_freq5)

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
for col in train_df.select_dtypes('object').columns:
    print(col, train_df[col].nunique())

In [ ]:
cols_to_remove = ['enrollee_id']
target = 'target'

In [ ]:
_X = train_df.drop(cols_to_remove + [target], axis=1)
y = train_df[[target]]
_XTEST = test_df.drop(cols_to_remove , axis=1)

In [ ]:
_X.shape, _XTEST.shape

In [ ]:
X_all = pd.concat([_X, _XTEST]).reset_index(drop=True)
X_all.shape

In [ ]:
# city 123
# gender 4
# relevent_experience 2
# enrolled_university 1
# education_level 1
# major_discipline 7
# experience 23
# company_size 9
# company_type 6
# last_new_job 7

In [ ]:
X_all.dtypes

In [ ]:
cat_columns = []
for col in X_all.select_dtypes('object').columns:
    print(col)
    cat_columns.append(col)
    le = LabelEncoder()
    X_all[col] = le.fit_transform(X_all[col])

In [ ]:
X_all.shape

In [ ]:
X = X_all[:len(y)]
XTEST = X_all[len(y):]
X.shape, XTEST.shape

In [ ]:
X.columns

In [ ]:
X.head()

In [ ]:
NUM_OF_BOOST_ROUND = 10000
EARLY_STOPPING = 300

In [ ]:
cat_features_index = [i for i,col in enumerate(X.columns) if col in cat_columns]
cat_features_index

In [ ]:
X_train , X_valid, y_train, y_valid = train_test_split(X,y, 
                                                       test_size=0.2, 
                                                       random_state=SEED, 
                                                       stratify=y)

LGBM Classifier 

In [ ]:
params = {
    'cat_features': cat_features_index,
    'eval_metric': 'AUC',
    'random_seed': SEED,
    'n_estimators': NUM_OF_BOOST_ROUND,
}

In [ ]:
clf = lgb.LGBMClassifier(**params, n_jobs= -1)

In [ ]:
clf.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=-1)

In [ ]:
ypred_lgb = clf.predict_proba(X_valid)[:,1]

In [ ]:
roc_auc_score(y_valid, ypred_lgb)

In [ ]:
ytest = clf.predict_proba(XTEST)[:,1]

In [ ]:
sub_df.Response = ytest

In [ ]:
sns.distplot(sub_df.Response)

CATBOOST Classifier 

In [ ]:
params = {
    'cat_features' : cat_features_index,
    'eval_metric': 'AUC',
    'random_seed': SEED,
    'n_estimators': NUM_OF_BOOST_ROUND
}

In [ ]:
bst = CatBoostClassifier(**params, early_stopping_rounds=EARLY_STOPPING,thread_count=-1)
_ = bst.fit(X_train, y_train, eval_set=(X_valid,y_valid), plot=True, verbose=False)

In [ ]:
ypred_cat = bst.predict_proba(X_valid)[:,1]

In [ ]:
roc_auc_score(y_valid, ypred_cat)

In [ ]:
f_importance_df = pd.DataFrame(bst.get_feature_importance(), columns=['importance'], index=X_valid.columns)
f_importance_df = f_importance_df.sort_values(by='importance', ascending=False)

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=f_importance_df.importance[:500], y=f_importance_df.index[:500]);

In [ ]:
ypred_test = bst.predict_proba(XTEST)[:,1]

In [ ]:
sns.distplot(ypred_test)

In [ ]:
pred_test_df = bst.predict(XTEST)

In [ ]:
sub_df

In [ ]:
sub_df['target'] = pred_test_df

In [ ]:
sub_df.target.value_counts()

In [ ]:
sub_df.to_csv("submission_hr.csv",index = False)

In [ ]:
def plot_roc(y_trues, y_preds, labels, x_max=1.0):
    fig, ax = plt.subplots()
    for i, y_pred in enumerate(y_preds):
        y_true = y_trues[i]
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred)
        ax.plot(fpr, tpr, label='%s; AUC=%.3f' % (labels[i], auc), marker='o', markersize=1)

    ax.legend()
    ax.grid()
    ax.plot(np.linspace(0, 1, 20), np.linspace(0, 1, 20), linestyle='--')
    ax.set_title('ROC curve')
    ax.set_xlabel('False Positive Rate')
    ax.set_xlim([-0.01, x_max])
    _ = ax.set_ylabel('True Positive Rate')

In [ ]:
plot_roc(
    [y_valid]*2,
    [ypred_lgb, ypred_cat],
    ['LGBM', 'CATBOOST'])